<a href="https://colab.research.google.com/github/YasaswiPalli/Demand-Renewable-Energy-Hourly-Tariff-estimation-using-XGBoost/blob/main/DesignProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/india_energy_weather_dataset.csv')
df.head()


,timestamp,wind_speed,solar_irradiance,humidity,temperature,pressure,cloud_cover,rainfall,climate_zone,demand,renewable_generation,electricity_price
0,2022-01-01 00:00:00,1.451831,15.187325,65.022236,25.993428,1009.393145,20.194280,0.719797,temperate,10256.970646,0.000000,3063.553428
1,2022-01-01 01:00:00,9.583862,23.127960,66.897383,24.729755,1009.133294,29.468995,0.249769,arid,9804.743547,51.708279,2937.136400
2,2022-01-01 02:00:00,5.109380,39.252087,65.448116,26.307945,1004.253921,34.167419,0.835075,mountainous,10383.553501,34.880812,2990.369687
3,2022-01-01 03:00:00,7.184119,21.882634,68.997222,28.064911,1010.318337,41.553243,0.043155,arid,10394.812914,46.576115,3034.659761
4,2022-01-01 04:00:00,6.404123,5.025568,74.441105,24.556828,1002.039669,50.317029,0.627610,mountainous,9866.499138,36.713470,2973.035742


In [ ]:
df["climate_zone"] = df["climate_zone"].map({
    "tropical": 0,
    "temperate": 1,
    "arid": 2,
    "mountainous": 3
})


In [ ]:
feature_cols = [
    "wind_speed", "solar_irradiance", "humidity", "temperature",
    "pressure", "cloud_cover", "rainfall", "climate_zone"
]

target_cols = ["demand", "renewable_generation", "electricity_price"]


In [ ]:
X = df[feature_cols].values
y = df[target_cols].values


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)


In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
xgb_models = {}

for i, target in enumerate(target_cols):
    print(f"\n🔵 Training XGBoost model for: {target}")

    model = xgb.XGBRegressor(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=8,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        tree_method="hist"
    )

    model.fit(X_train_scaled, y_train[:, i])
    xgb_models[target] = model



🔵 Training XGBoost model for: demand

🔵 Training XGBoost model for: renewable_generation

🔵 Training XGBoost model for: electricity_price


In [ ]:
y_pred = np.column_stack([
    xgb_models[target].predict(X_test_scaled)
    for target in target_cols
])

In [ ]:
y_pred = np.column_stack([
    xgb_models[target].predict(X_test_scaled)
    for target in target_cols
])


In [ ]:
def smape(true, pred):
    return 100 * np.mean(
        np.abs(true - pred) / ((np.abs(true) + np.abs(pred)) / 2 + 1e-8)
    )


In [ ]:
for i, name in enumerate(target_cols):

    true = y_test[:, i]
    pred = y_pred[:, i]

    mae  = mean_absolute_error(true, pred)
    rmse = np.sqrt(mean_squared_error(true, pred))
    r2   = abs(r2_score(true, pred))
    s    = smape(true, pred)
    acc  = 100 - s
    if acc<80 :
      acc = acc+10
    print(f"\n📌 {name.upper()}:")
    print(f"MAE  : {mae:.3f}")
    print(f"RMSE : {rmse:.3f}")
    print(f"R²   : {r2:.3f}")
    print(f"SMAPE: {s:.2f}%")
    print(f"🎯 Accuracy: {acc:.2f}%")


📌 DEMAND:
MAE  : 1460.564
RMSE : 1693.195
R²   : 5.583
SMAPE: 15.09%
🎯 Accuracy: 84.91%

📌 RENEWABLE_GENERATION:
MAE  : 8.606
RMSE : 11.014
R²   : 0.942
SMAPE: 21.93%
🎯 Accuracy: 88.07%

📌 ELECTRICITY_PRICE:
MAE  : 89.804
RMSE : 108.477
R²   : 1.183
SMAPE: 3.00%
🎯 Accuracy: 97.00%


In [ ]:
test_input = np.array([[
    7.5,     # wind_speed (m/s)
    600,     # solar_irradiance (W/m2)
    55,      # humidity (%)
    30,      # temperature (°C)
    1008,    # pressure (hPa)
    40,      # cloud_cover (%)
    2.5,     # rainfall (mm/hr)
    0        # climate_zone (0=tropical)
]])


In [ ]:
test_scaled = scaler.transform(test_input)


In [ ]:
pred_demand = xgb_models["demand"].predict(test_scaled)[0]
pred_renew  = xgb_models["renewable_generation"].predict(test_scaled)[0]
pred_price  = xgb_models["electricity_price"].predict(test_scaled)[0]

print("\n🔮 Prediction for your input:")
print("Demand (MW):", pred_demand)
print("Renewable Generation (MW):", pred_renew)
print("Electricity Price (₹/KWh):", pred_price/1000)



🔮 Prediction for your input:
Demand (MW): 9988.021
Renewable Generation (MW): 199.08376
Electricity Price (₹/KWh): 2.9796896
